In [1]:
# !/bin/python3
# https://adventofcode.com/2022/day/15

%load_ext lab_black

In [2]:
import math
import re

from collections import defaultdict
from utils import read_input

pattern = re.compile("x=(-?\d+), y=(-?\d+): closest beacon is at x=(-?\d+), y=(-?\d+)")

In [3]:
def parse(line):
    match = pattern.search(line).groups(0)
    return [(int(match[1]), int(match[0])), (int(match[3]), int(match[2]))]


def make_map(sensors):
    data = {}
    for sensor, beacon in sensors:
        distance = abs(sensor[0] - beacon[0]) + abs(sensor[1] - beacon[1])
        data[sensor] = {"beacon": beacon, "distance": distance}
    return data

In [4]:
def get_impossible_beacon_locations(sensors, row):
    min_column, max_column = get_map_edges(sensors)
    impossible = get_sensor_ranges(sensors, row, min_column, max_column)

    return impossible


def get_map_edges(sensors):
    columns = set()
    for sensor, data in sensors.items():
        columns.add(sensor[1])
        columns.add(data["beacon"][1])
    return min(columns), max(columns)


def get_sensor_ranges(sensors, row, min_column, max_column):
    impossible = set()
    for sensor, data in sensors.items():
        distance = data["distance"] - abs(sensor[0] - row)

        if distance < 0:
            continue

        left_column = sensor[1] - distance
        right_column = sensor[1] + distance

        impossible.update(range(left_column, right_column + 1))
        if data["beacon"][0] == row and data["beacon"][1] in impossible:
            impossible.remove(data["beacon"][1])

    return impossible

In [5]:
def part_a(sensors, row):
    return len(get_impossible_beacon_locations(sensors, row))

In [6]:
def get_sensor_scan_areas(sensors, min_coor, max_coor):
    areas = {}

    for sensor, data in sensors.items():
        row = max(min_coor, min(max_coor, sensor[0]))
        column = max(min_coor, min(max_coor, sensor[1]))
        areas[sensor] = {
            "distance": data["distance"],
            "top": (sensor[0] - data["distance"] - 1, sensor[1]),
            "bottom": (sensor[0] + data["distance"] + 1, sensor[1]),
            "left": (sensor[0], sensor[1] - data["distance"] - 1),
            "right": (sensor[0], sensor[1] + data["distance"] + 1),
        }

    return areas


def get_distance(sensor, beacon):
    return abs(sensor[0] - beacon[0]) + abs(sensor[1] - beacon[1])


def check_bounds(beacon, min_coor, max_coor):
    if beacon[0] < min_coor or beacon[0] > max_coor:
        return False
    if beacon[1] < min_coor or beacon[1] > max_coor:
        return False
    return True


def move_inbound(beacon, min_coor, max_coor, offset):
    if beacon[0] < min_coor:
        beacon = (
            beacon[0] + (offset[0] * (min_coor - beacon[0])),
            beacon[1] + (offset[1] * (min_coor - beacon[0])),
        )
    if beacon[0] > max_coor:
        beacon = (
            beacon[0] + (offset[0] * (beacon[0] - max_coor)),
            beacon[1] + (offset[1] * (beacon[0] - max_coor)),
        )
    if beacon[1] < min_coor:
        beacon = (
            beacon[0] + (offset[0] * (min_coor - beacon[1])),
            beacon[1] + (offset[1] * (min_coor - beacon[1])),
        )
    if beacon[1] > max_coor:
        beacon = (
            beacon[0] + (offset[0] * (beacon[1] - max_coor)),
            beacon[1] + (offset[1] * (beacon[1] - max_coor)),
        )
    return beacon


def move_out_of_sensor_range(beacon, diff, offset):
    diff = math.ceil((diff + 1) / 2)
    return (
        beacon[0] + (offset[0] * diff),
        beacon[1] + (offset[1] * diff),
    )


def check_boundary(sensors, sensor, data, min_coor, max_coor):
    corners = (data["top"], data["bottom"], data["left"], data["right"])
    diffs = ((1, 1), (-1, -1), (-1, 1), (-1, 1))
    for corner, offset in zip(corners, diffs):
        beacon = corner
        if not check_bounds(beacon, min_coor, max_coor):
            beacon = move_inbound(beacon, min_coor, max_coor, offset)

        if not check_bounds(beacon, min_coor, max_coor):
            continue

        while get_distance(sensor, beacon) <= data["distance"] + 2 and check_bounds(
            beacon, min_coor, max_coor
        ):
            good = True
            for check_sensor, check_data in sensors.items():
                if sensor == check_sensor:
                    continue

                curr_distance = get_distance(check_sensor, beacon)
                while curr_distance <= check_data["distance"]:
                    good = False
                    beacon = move_out_of_sensor_range(
                        beacon, check_data["distance"] - curr_distance, offset
                    )
                    curr_distance = get_distance(check_sensor, beacon)

                    if get_distance(sensor, beacon) > data[
                        "distance"
                    ] + 2 or not check_bounds(beacon, min_coor, max_coor):
                        break

                if not good:
                    break
            if good:
                return beacon

In [7]:
def part_b(sensors, min_coor, max_coor):
    areas = get_sensor_scan_areas(sensors, min_coor, max_coor)
    for sensor, data in areas.items():
        result = check_boundary(sensors, sensor, data, min_coor, max_coor)
        if result:
            return result[1] * 4_000_000 + result[0]

In [8]:
sensors = make_map(read_input(parent=__vsc_ipynb_file__, val_type=parse, sample="a"))

print("part a:", part_a(sensors, 10))
print("part b:", part_b(sensors, 0, 20))

part a: 26
part b: 56000011


In [9]:
sensors = make_map(read_input(parent=__vsc_ipynb_file__, val_type=parse))

print("part a:", part_a(sensors, 2_000_000))
print("part b:", part_b(sensors, 0, 4_000_000))

part a: 5832528
part b: 13360899249595
